# ライブラリの読み込み

In [1]:
import numpy as np
import pandas as pd
import regex
from tqdm import tqdm
import warnings
import six
from google.cloud import translate_v2 as translate
import enchant

tqdm.pandas()
warnings.filterwarnings('ignore')

# 問題データの読み込み

In [2]:
path_quiz = "../input/abcthe19th_EQIDEN2021問題_ジャンル付き.xlsx"
df_quiz = pd.read_excel(path_quiz, sheet_name=None)
print(df_quiz.keys())

dict_keys(['abc四択', 'abc筆記', 'abc本戦', 'abc敗者復活1st', 'EQIDEN'])


In [3]:
df_quiz1 = df_quiz['abc四択']
print(df_quiz1.shape)
df_quiz1.head()

(100, 9)


,No.,大ジャンル,問題文,答え,正解選択肢,選択肢1,選択肢2,選択肢3,選択肢4
0,1,ns,物事のランク付けで「A」より上のものを指して使われることがある、アルファベット19番目の文字は？,S,3,Q,R,S,T
1,2,生活,「木綿」や「絹ごし」などの種類がある、大豆から作られる食品は？,豆腐,1,豆腐,納豆,醤油,味噌
2,3,生活,視力を補正するためではなく、おしゃれのためにかけるめがねを「何めがね」という？,ダテめがね,2,ガチ,ダテ,ユル,ギリ
3,4,漫アゲ,昨年（2020年）11月12日に発売された、SONYの最新家庭用ゲーム機は？,PlayStation 5,3,PlayStation 3,PlayStation 4,PlayStation 5,PlayStation 6
4,5,理系,正五角形の一辺と対角線の長さの比にも表れる、人間が美しいと感じるおよそ1:1.618の比を何...,黄金比,1,黄金比,白銀比,赤銅比,真鍮比


# データの加工

In [4]:
df_quiz1_1 = df_quiz1.copy()
df_quiz1_1 = df_quiz1_1[['No.', '問題文', '答え']]
df_quiz1_1['id'] = df_quiz1_1['No.'].apply(lambda x: "abcthe19th_abc四択_{}".format(x))
df_quiz1_1['question'] = df_quiz1_1['問題文']
df_quiz1_1['answer'] = df_quiz1_1['答え']
df_quiz1_1 = df_quiz1_1[['id', 'question', 'answer']]
df_quiz1_1.tail()

,id,question,answer
95,abcthe19th_abc四択_96,イリヤ・レーピンやイワン・クラムスコイに代表されるロシアの画家集団を、国内各地で美術展を開い...,移動派
96,abcthe19th_abc四択_97,パルプの原料や燃料などに用いられる、サトウキビを搾った後に残るカスを何という？,バガス
97,abcthe19th_abc四択_98,ゴルフで、ボギー以下の結果となった直後のホールでバーディ以上の結果を出した確率のことを何という？,バウンスバック率
98,abcthe19th_abc四択_99,林家今丸、林家二楽、林家正楽といえば、どんな伝統芸能の芸人？,紙切り
99,abcthe19th_abc四択_100,古文書などに書かれた崩し字を読み取り、現代の文字に置き換える作業のことを漢字２文字で何という？,翻刻（ほんこく）


# 解答のフリガナを削除

In [5]:
def func1(x):
    x = str(x)
    tmp = x.split('（')[0]
    return tmp

df_quiz1_1['answer_trim'] = df_quiz1_1['answer'].apply(lambda x: func1(x))
df_quiz1_1.tail()

,id,question,answer,answer_trim
95,abcthe19th_abc四択_96,イリヤ・レーピンやイワン・クラムスコイに代表されるロシアの画家集団を、国内各地で美術展を開い...,移動派,移動派
96,abcthe19th_abc四択_97,パルプの原料や燃料などに用いられる、サトウキビを搾った後に残るカスを何という？,バガス,バガス
97,abcthe19th_abc四択_98,ゴルフで、ボギー以下の結果となった直後のホールでバーディ以上の結果を出した確率のことを何という？,バウンスバック率,バウンスバック率
98,abcthe19th_abc四択_99,林家今丸、林家二楽、林家正楽といえば、どんな伝統芸能の芸人？,紙切り,紙切り
99,abcthe19th_abc四択_100,古文書などに書かれた崩し字を読み取り、現代の文字に置き換える作業のことを漢字２文字で何という？,翻刻（ほんこく）,翻刻


# 解答が漢字のみで構成されるか判定

In [6]:
def func2(x):
    p = regex.compile(r'\p{Script=Han}+')
    tmp = p.fullmatch(x)
    if tmp==None:
        ans = 0
    else:
        ans = 1
    return ans

df_quiz1_1['answer_han'] = df_quiz1_1['answer_trim'].apply(lambda x: func2(x))
df_quiz1_1.tail()

,id,question,answer,answer_trim,answer_han
95,abcthe19th_abc四択_96,イリヤ・レーピンやイワン・クラムスコイに代表されるロシアの画家集団を、国内各地で美術展を開い...,移動派,移動派,1
96,abcthe19th_abc四択_97,パルプの原料や燃料などに用いられる、サトウキビを搾った後に残るカスを何という？,バガス,バガス,0
97,abcthe19th_abc四択_98,ゴルフで、ボギー以下の結果となった直後のホールでバーディ以上の結果を出した確率のことを何という？,バウンスバック率,バウンスバック率,0
98,abcthe19th_abc四択_99,林家今丸、林家二楽、林家正楽といえば、どんな伝統芸能の芸人？,紙切り,紙切り,0
99,abcthe19th_abc四択_100,古文書などに書かれた崩し字を読み取り、現代の文字に置き換える作業のことを漢字２文字で何という？,翻刻（ほんこく）,翻刻,1


In [7]:
# 答えが漢字のみの問題を抽出
df_quiz1_2 = df_quiz1_1[df_quiz1_1['answer_han']==1]
print(df_quiz1_2.shape)
df_quiz1_2.head()

(31, 5)


,id,question,answer,answer_trim,answer_han
1,abcthe19th_abc四択_2,「木綿」や「絹ごし」などの種類がある、大豆から作られる食品は？,豆腐,豆腐,1
4,abcthe19th_abc四択_5,正五角形の一辺と対角線の長さの比にも表れる、人間が美しいと感じるおよそ1:1.618の比を何...,黄金比,黄金比,1
8,abcthe19th_abc四択_9,一周忌の１年後、人が亡くなって２年経った際に開かれる法事は？,三回忌,三回忌,1
10,abcthe19th_abc四択_11,ダリの絵画『記憶の固執』で、ぐにゃりと溶けたように描かれているものは？,時計,時計,1
12,abcthe19th_abc四択_13,五大堂や瑞巌寺といった史跡も位置する、日本三景の一つに数えられる宮城県の名勝地は？,松島,松島,1


# 上記の処理を全てのシートに対して実施

In [8]:
quiz_name = 'abc19'
df_all = pd.DataFrame()
tmp_keys = df_quiz.keys()
print(tmp_keys)
for key in tmp_keys:
    df_tmp = df_quiz[key]
    print("sheet:. {}".format(key))
    print("columns: {}".format(df_tmp.columns.values))
#     print(df_tmp.head())
    df_tmp2 = df_tmp[['No.', '問題文', '答え']]
    df_tmp2['id'] = df_tmp2['No.'].apply(lambda x: "{}_{}_{}".format(quiz_name, key, x))
    df_tmp2['question'] = df_tmp2['問題文']
    df_tmp2['answer'] = df_tmp2['答え']
    df_tmp2 = df_tmp2[['id', 'question', 'answer']]
    df_tmp2['answer_trim'] = df_tmp2['answer'].apply(lambda x: func1(x))
    df_tmp2['answer_han'] = df_tmp2['answer_trim'].apply(lambda x: func2(x))
    df_tmp2 = df_tmp2[df_tmp2['answer_han']==1].reset_index(drop=True)
    print(df_tmp2.shape)
    df_all = pd.concat([df_all, df_tmp2]).reset_index(drop=True)
print(df_all.shape)
df_all.head()

dict_keys(['abc四択', 'abc筆記', 'abc本戦', 'abc敗者復活1st', 'EQIDEN'])
sheet:. abc四択
columns: ['No.' '大ジャンル' '問題文' '答え' '正解選択肢' '選択肢1' '選択肢2' '選択肢3' '選択肢4']
(31, 5)
sheet:. abc筆記
columns: ['No.' '大ジャンル' '小ジャンル' '問題文' '答え' '別解・正誤判定基準']
(35, 5)
sheet:. abc本戦
columns: ['No.' '大ジャンル' '小ジャンル' '問題文' '答え' '別解・正誤判定基準']
(284, 5)
sheet:. abc敗者復活1st
columns: ['No.' '大ジャンル' '小ジャンル' '問題文' '答え' '別解・正誤判定基準']
(8, 5)
sheet:. EQIDEN
columns: ['No.' '大ジャンル' '小ジャンル' '問題文' '答え' '別解・正誤判定基準']
(131, 5)
(489, 5)


,id,question,answer,answer_trim,answer_han
0,abc19_abc四択_2,「木綿」や「絹ごし」などの種類がある、大豆から作られる食品は？,豆腐,豆腐,1
1,abc19_abc四択_5,正五角形の一辺と対角線の長さの比にも表れる、人間が美しいと感じるおよそ1:1.618の比を何...,黄金比,黄金比,1
2,abc19_abc四択_9,一周忌の１年後、人が亡くなって２年経った際に開かれる法事は？,三回忌,三回忌,1
3,abc19_abc四択_11,ダリの絵画『記憶の固執』で、ぐにゃりと溶けたように描かれているものは？,時計,時計,1
4,abc19_abc四択_13,五大堂や瑞巌寺といった史跡も位置する、日本三景の一つに数えられる宮城県の名勝地は？,松島,松島,1


# グーグル翻訳の実行

In [9]:
# Google Translate APIのテスト
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    return result["translatedText"]

tmp = translate_text("en","私はペンです")
print(tmp)

I am a pen


In [10]:
# 一文字ずつ英訳するテスト
def func3(x):
    ans = ""
    for i in range(len(x)):
        tmp = translate_text('en', x[i])
        ans += tmp + " "
    ans = ans.strip()
    ans = ans.lower()
    return ans


tmp_ans = df_all['answer_trim'][0]
print(tmp_ans)
tmp = func3(tmp_ans)
print(tmp)

豆腐
beans corruption


In [11]:
# 4文字以上の答えの問題を抽出
df_all['len_han'] = df_all['answer_trim'].apply(lambda x: len(x))
df_all2 = df_all[df_all['len_han']>=4].reset_index(drop=True)
print(df_all2.shape)
df_all2.head()

(190, 6)


,id,question,answer,answer_trim,answer_han,len_han
0,abc19_abc四択_37,日本神話で、スサノオがヤマタノオロチを退治した際にその尻尾から出てきたという三種の神器のひとつは？,天叢雲剣（あめのむらくものつるぎ）,天叢雲剣,1,4
1,abc19_abc四択_38,カップ麺の「赤いきつね」や「緑のたぬき」を製造・販売している企業は？,東洋水産,東洋水産,1,4
2,abc19_abc四択_42,1980年代に『少女A』や『DESIRE -情熱-』などの楽曲を歌いヒットを飛ばした女性歌手は？,中森明菜（なかもり・あきな）,中森明菜,1,4
3,abc19_abc四択_52,ルソーの『社会契約論』を翻訳した『民約訳解（みんやくやくかい）』を著し、「東洋のルソー」と呼...,中江兆民（なかえ・ちょうみん）,中江兆民,1,4
4,abc19_abc四択_74,戦場での体験を基に、『俘虜記（ふりょき）』『野火（のび）』『レイテ戦記』などの戦争文学を著し...,大岡昇平（おおおか・しょうへい）,大岡昇平,1,4


In [12]:
# 答えを英訳
df_all2['translate'] = df_all2['answer_trim'].progress_apply(lambda x: func3(x))
df_all2.head()

100%|████████████████████████████████████████████████████████████████████████████████| 190/190 [06:49<00:00,  2.16s/it]


,id,question,answer,answer_trim,answer_han,len_han,translate
0,abc19_abc四択_37,日本神話で、スサノオがヤマタノオロチを退治した際にその尻尾から出てきたという三種の神器のひとつは？,天叢雲剣（あめのむらくものつるぎ）,天叢雲剣,1,4,sky cluster cloud sword
1,abc19_abc四択_38,カップ麺の「赤いきつね」や「緑のたぬき」を製造・販売している企業は？,東洋水産,東洋水産,1,4,east foreign water production
2,abc19_abc四択_42,1980年代に『少女A』や『DESIRE -情熱-』などの楽曲を歌いヒットを飛ばした女性歌手は？,中森明菜（なかもり・あきな）,中森明菜,1,4,middle forest bright vegetable
3,abc19_abc四択_52,ルソーの『社会契約論』を翻訳した『民約訳解（みんやくやくかい）』を著し、「東洋のルソー」と呼...,中江兆民（なかえ・ちょうみん）,中江兆民,1,4,middle river trillion civil
4,abc19_abc四択_74,戦場での体験を基に、『俘虜記（ふりょき）』『野火（のび）』『レイテ戦記』などの戦争文学を著し...,大岡昇平（おおおか・しょうへい）,大岡昇平,1,4,big hill rising flat


# 英訳した答えの単語数をカウントし日本語の文字数と一致するか確認

In [13]:
def func5(x):
    ans = x.split()
    return len(ans)

df_all2['len_en'] = df_all2['translate'].apply(lambda x: func5(x))
df_all2['match'] = df_all2['len_en']==df_all2['len_han']
print(df_all2.shape)
print(df_all2['match'].sum())
df_all3 = df_all2[df_all2['match']==True]
print(df_all3.shape)
df_all3.head()

(190, 9)
184
(184, 9)


,id,question,answer,answer_trim,answer_han,len_han,translate,len_en,match
0,abc19_abc四択_37,日本神話で、スサノオがヤマタノオロチを退治した際にその尻尾から出てきたという三種の神器のひとつは？,天叢雲剣（あめのむらくものつるぎ）,天叢雲剣,1,4,sky cluster cloud sword,4,True
1,abc19_abc四択_38,カップ麺の「赤いきつね」や「緑のたぬき」を製造・販売している企業は？,東洋水産,東洋水産,1,4,east foreign water production,4,True
2,abc19_abc四択_42,1980年代に『少女A』や『DESIRE -情熱-』などの楽曲を歌いヒットを飛ばした女性歌手は？,中森明菜（なかもり・あきな）,中森明菜,1,4,middle forest bright vegetable,4,True
3,abc19_abc四択_52,ルソーの『社会契約論』を翻訳した『民約訳解（みんやくやくかい）』を著し、「東洋のルソー」と呼...,中江兆民（なかえ・ちょうみん）,中江兆民,1,4,middle river trillion civil,4,True
4,abc19_abc四択_74,戦場での体験を基に、『俘虜記（ふりょき）』『野火（のび）』『レイテ戦記』などの戦争文学を著し...,大岡昇平（おおおか・しょうへい）,大岡昇平,1,4,big hill rising flat,4,True


# 英訳が上手くいっているかの確認のテスト

In [14]:
d = enchant.Dict("en_US") # 英語の辞書を読み込み
tmp = "hello"
print("{}: {}".format(tmp, d.check(tmp))) # 英語の辞書にある単語かの判定
tmp = "ohayo"
print("{}: {}".format(tmp, d.check(tmp))) # 英語の辞書にある単語かの判定

hello: True
ohayo: False


In [15]:
# 英訳が上手くいっているかの確認
def func6(x):
    words = x.split(" ")
    for word in words:
        tmp = d.check(word)
        if tmp==False:
            return False
    return True


df_all3['en_ok'] = df_all3['translate'].apply(lambda x: func6(x))
print(df_all3.shape)
print(df_all3['en_ok'].sum())
df_all3.tail()

(184, 10)
127


,id,question,answer,answer_trim,answer_han,len_han,translate,len_en,match,en_ok
185,abc19_EQIDEN_311,昨年（2020年）公開の『事故物件 怖い間取り』や、『仄暗い水の底から』『リング』といったホ...,中田秀夫（なかた・ひでお）,中田秀夫,1,4,middle field show husband,4,True,True
186,abc19_EQIDEN_313,1958年にシャルル・ド・ゴールを中心に樹立され現在まで続く、フランスの政治体制は「第何共和...,第五共和政,第五共和政,1,5,no. five common and politics,5,True,True
187,abc19_EQIDEN_339,名探偵の由利麟太郎や金田一耕助の生みの親である小説家は誰でしょう？,横溝正史（よこみぞ・せいし）,横溝正史,1,4,beside ditch just history,4,True,True
188,abc19_EQIDEN_346,シアトル・マリナーズの菊池雄星と、ロサンゼルス・エンゼルスの大谷翔平に共通する出身高校はどこ...,花巻東高校,花巻東高校,1,5,flower volume east high school,5,True,True
189,abc19_EQIDEN_349,故郷・安曇野市の碌山（ろくざん）美術館に作品が収められている、『女』などの代表作がある彫刻家...,荻原守衛（おぎわら・もりえ）,荻原守衛,1,4,ogi original guard guard,4,True,False


In [16]:
# 英訳がうまくいっている問題だけ抽出
df_all4 = df_all3[df_all3['en_ok']].reset_index(drop=True)
print(df_all4.shape)
df_all4.tail()

(127, 10)


,id,question,answer,answer_trim,answer_han,len_han,translate,len_en,match,en_ok
122,abc19_EQIDEN_298,多くの人が感染症の抗体を獲得することで感染拡大が抑えられる状態のことを、漢字４文字で何という...,集団免疫,集団免疫,1,4,set group waived epidemic,4,True,True
123,abc19_EQIDEN_311,昨年（2020年）公開の『事故物件 怖い間取り』や、『仄暗い水の底から』『リング』といったホ...,中田秀夫（なかた・ひでお）,中田秀夫,1,4,middle field show husband,4,True,True
124,abc19_EQIDEN_313,1958年にシャルル・ド・ゴールを中心に樹立され現在まで続く、フランスの政治体制は「第何共和...,第五共和政,第五共和政,1,5,no. five common and politics,5,True,True
125,abc19_EQIDEN_339,名探偵の由利麟太郎や金田一耕助の生みの親である小説家は誰でしょう？,横溝正史（よこみぞ・せいし）,横溝正史,1,4,beside ditch just history,4,True,True
126,abc19_EQIDEN_346,シアトル・マリナーズの菊池雄星と、ロサンゼルス・エンゼルスの大谷翔平に共通する出身高校はどこ...,花巻東高校,花巻東高校,1,5,flower volume east high school,5,True,True


# データの保存

In [17]:
# データベースを成型
df_all5 = pd.DataFrame(np.arange(len(df_all4)), columns=['id'])
df_all5['question'] = df_all4['translate']
df_all5['answer'] = df_all4['answer']
df_all5.head()

,id,question,answer
0,0,sky cluster cloud sword,天叢雲剣（あめのむらくものつるぎ）
1,1,east foreign water production,東洋水産
2,2,middle forest bright vegetable,中森明菜（なかもり・あきな）
3,3,middle river trillion civil,中江兆民（なかえ・ちょうみん）
4,4,big hill rising flat,大岡昇平（おおおか・しょうへい）


In [18]:
# 保存
df_all5.to_csv("../output/FirstSummerUikaQuiz.csv", index=None, encoding='cp932')

# 出題テスト

In [24]:
i = np.random.randint(len(df_all5))
print("問題\n{}\n答え\n↓\n↓\n↓\n↓\n↓\n↓\n↓\n↓\n↓\n↓\n{}".format(df_all5['question'][i], df_all5['answer'][i]))

問題
can see light wire
答え
↓
↓
↓
↓
↓
↓
↓
↓
↓
↓
可視光線
